Evaluate Flair models for person names, orgs and locations using the Presidio Evaluator framework

Data = `test_February_28_2020`

Note: *Make sure to run the notebook `Create datasets for Spacy training.ipynb` as the data generated is required for this notebook*

In [1]:
! cd ../../models && python flair_train.py

Kept 194 train samples after removal of non-tagged samples
2020-03-24 00:38:21,145 Reading data from .
2020-03-24 00:38:21,145 Train: flair_train.txt
2020-03-24 00:38:21,145 Dev: flair_test.txt
2020-03-24 00:38:21,145 Test: flair_val.txt
Corpus: 194 train + 59 dev + 28 test sentences
[b'<unk>', b'O', b'B-PERSON', b'I-PERSON', b'B-ORG', b'I-ORG', b'B-GPE', b'I-GPE', b'<START>', b'<STOP>']
2020-03-24 00:38:21,997 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings/glove.gensim.vectors.npy not found in cache, downloading to /tmp/tmpwljfty72
100%|█████████████████████████| 160000128/160000128 [01:07<00:00, 2371694.10B/s]
2020-03-24 00:39:30,173 copying /tmp/tmpwljfty72 to cache at /home/jovyan/.flair/embeddings/glove.gensim.vectors.npy
2020-03-24 00:39:30,409 removing temp file /tmp/tmpwljfty72
2020-03-24 00:39:31,103 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings/glove.gensim not found in cache, downloading to /tmp/tmp_5n_7rt1
100%|████████████████████

2020-03-24 00:55:37,763 DEV : loss 2.8567967414855957 - score 0.5667
2020-03-24 00:55:37,780 BAD EPOCHS (no improvement): 1
2020-03-24 00:55:49,264 ----------------------------------------------------------------------------------------------------
2020-03-24 00:55:53,984 epoch 13 - iter 1/7 - loss 1.06870484 - samples/sec: 6.78
2020-03-24 00:55:58,464 epoch 13 - iter 2/7 - loss 1.20885062 - samples/sec: 7.17
2020-03-24 00:56:02,842 epoch 13 - iter 3/7 - loss 1.25085505 - samples/sec: 7.36
2020-03-24 00:56:07,246 epoch 13 - iter 4/7 - loss 1.31458139 - samples/sec: 7.30
2020-03-24 00:56:11,520 epoch 13 - iter 5/7 - loss 1.69409704 - samples/sec: 7.53
2020-03-24 00:56:16,130 epoch 13 - iter 6/7 - loss 1.76311660 - samples/sec: 6.97
2020-03-24 00:56:16,992 epoch 13 - iter 7/7 - loss 1.59736286 - samples/sec: 38.16
2020-03-24 00:56:17,018 ----------------------------------------------------------------------------------------------------
2020-03-24 00:56:17,018 EPOCH 13 done: loss 1.5974 

2020-03-24 01:01:06,300 ----------------------------------------------------------------------------------------------------
2020-03-24 01:01:10,002 epoch 21 - iter 1/7 - loss 0.77585113 - samples/sec: 8.65
2020-03-24 01:01:13,868 epoch 21 - iter 2/7 - loss 0.62576240 - samples/sec: 8.32
2020-03-24 01:01:17,953 epoch 21 - iter 3/7 - loss 0.70086563 - samples/sec: 7.87
2020-03-24 01:01:21,946 epoch 21 - iter 4/7 - loss 0.83926824 - samples/sec: 8.05
2020-03-24 01:01:26,135 epoch 21 - iter 5/7 - loss 0.74448411 - samples/sec: 7.68
2020-03-24 01:01:29,917 epoch 21 - iter 6/7 - loss 0.77081014 - samples/sec: 8.51
2020-03-24 01:01:30,623 epoch 21 - iter 7/7 - loss 0.67290034 - samples/sec: 47.06
2020-03-24 01:01:30,648 ----------------------------------------------------------------------------------------------------
2020-03-24 01:01:30,648 EPOCH 21 done: loss 0.6729 - lr 0.0500
2020-03-24 01:01:32,702 DEV : loss 1.827229619026184 - score 0.7213
2020-03-24 01:01:32,710 BAD EPOCHS (no impro

2020-03-24 01:06:06,562 epoch 29 - iter 1/7 - loss 0.50206953 - samples/sec: 8.27
2020-03-24 01:06:08,892 epoch 29 - iter 2/7 - loss 0.41867703 - samples/sec: 13.85
2020-03-24 01:06:13,281 epoch 29 - iter 3/7 - loss 0.40759162 - samples/sec: 7.32
2020-03-24 01:06:18,227 epoch 29 - iter 4/7 - loss 0.43191963 - samples/sec: 6.50
2020-03-24 01:06:22,586 epoch 29 - iter 5/7 - loss 0.52025236 - samples/sec: 7.39
2020-03-24 01:06:27,368 epoch 29 - iter 6/7 - loss 0.51745047 - samples/sec: 6.73
2020-03-24 01:06:28,194 epoch 29 - iter 7/7 - loss 0.52306841 - samples/sec: 40.19
2020-03-24 01:06:28,224 ----------------------------------------------------------------------------------------------------
2020-03-24 01:06:28,224 EPOCH 29 done: loss 0.5231 - lr 0.0500
2020-03-24 01:06:31,229 DEV : loss 1.68401038646698 - score 0.748
2020-03-24 01:06:31,247 BAD EPOCHS (no improvement): 1
2020-03-24 01:06:41,750 -------------------------------------------------------------------------------------------

2020-03-24 01:11:38,157 epoch 37 - iter 1/7 - loss 0.36193049 - samples/sec: 9.19
2020-03-24 01:11:42,490 epoch 37 - iter 2/7 - loss 0.37851274 - samples/sec: 7.42
2020-03-24 01:11:44,996 epoch 37 - iter 3/7 - loss 0.34751485 - samples/sec: 12.88
2020-03-24 01:11:47,336 epoch 37 - iter 4/7 - loss 0.31360863 - samples/sec: 13.79
2020-03-24 01:11:51,423 epoch 37 - iter 5/7 - loss 0.34480156 - samples/sec: 7.87
2020-03-24 01:11:55,746 epoch 37 - iter 6/7 - loss 0.44630310 - samples/sec: 7.44
2020-03-24 01:11:56,465 epoch 37 - iter 7/7 - loss 0.39417898 - samples/sec: 45.77
2020-03-24 01:11:56,485 ----------------------------------------------------------------------------------------------------
2020-03-24 01:11:56,485 EPOCH 37 done: loss 0.3942 - lr 0.0125
2020-03-24 01:11:58,390 DEV : loss 1.7610716819763184 - score 0.7769
2020-03-24 01:11:58,398 BAD EPOCHS (no improvement): 1
2020-03-24 01:12:19,869 ---------------------------------------------------------------------------------------

2020-03-24 01:21:53,737 epoch 53 - iter 2/7 - loss 0.42435330 - samples/sec: 7.15
2020-03-24 01:21:57,995 epoch 53 - iter 3/7 - loss 0.39243662 - samples/sec: 7.55
2020-03-24 01:22:02,885 epoch 53 - iter 4/7 - loss 0.39270137 - samples/sec: 6.58
2020-03-24 01:22:07,849 epoch 53 - iter 5/7 - loss 0.37310467 - samples/sec: 6.47
2020-03-24 01:22:13,386 epoch 53 - iter 6/7 - loss 0.38702097 - samples/sec: 5.81
2020-03-24 01:22:14,604 epoch 53 - iter 7/7 - loss 0.41034717 - samples/sec: 26.88
2020-03-24 01:22:14,643 ----------------------------------------------------------------------------------------------------
2020-03-24 01:22:14,643 EPOCH 53 done: loss 0.4103 - lr 0.0016
2020-03-24 01:22:16,860 DEV : loss 1.6784909963607788 - score 0.7769
2020-03-24 01:22:16,871 BAD EPOCHS (no improvement): 1
2020-03-24 01:22:24,081 ----------------------------------------------------------------------------------------------------
2020-03-24 01:22:28,487 epoch 54 - iter 1/7 - loss 0.57240927 - sample

2020-03-24 01:27:02,544 epoch 61 - iter 2/7 - loss 0.45837921 - samples/sec: 9.86
2020-03-24 01:27:07,419 epoch 61 - iter 3/7 - loss 0.38786316 - samples/sec: 6.60
2020-03-24 01:27:12,415 epoch 61 - iter 4/7 - loss 0.39905179 - samples/sec: 6.45
2020-03-24 01:27:17,116 epoch 61 - iter 5/7 - loss 0.42342687 - samples/sec: 6.84
2020-03-24 01:27:22,071 epoch 61 - iter 6/7 - loss 0.42498639 - samples/sec: 6.49
2020-03-24 01:27:23,149 epoch 61 - iter 7/7 - loss 0.36524407 - samples/sec: 30.54
2020-03-24 01:27:23,185 ----------------------------------------------------------------------------------------------------
2020-03-24 01:27:23,186 EPOCH 61 done: loss 0.3652 - lr 0.0004
2020-03-24 01:27:26,140 DEV : loss 1.6660104990005493 - score 0.7769
2020-03-24 01:27:26,153 BAD EPOCHS (no improvement): 1
2020-03-24 01:27:32,560 ----------------------------------------------------------------------------------------------------
2020-03-24 01:27:37,474 epoch 62 - iter 1/7 - loss 0.57466054 - sample

2020-03-24 01:32:22,647 ----------------------------------------------------------------------------------------------------
2020-03-24 01:32:22,647 Testing using best model ...
2020-03-24 01:32:22,647 loading file resources/taggers/presidio-ner/best-model.pt
2020-03-24 01:32:40,201 0.8235	0.7	0.7567
2020-03-24 01:32:40,201 
MICRO_AVG: acc 0.6087 - f1-score 0.7567
MACRO_AVG: acc 0.5683 - f1-score 0.6778
GPE        tp: 6 - fp: 2 - fn: 2 - tn: 6 - precision: 0.7500 - recall: 0.7500 - accuracy: 0.6000 - f1-score: 0.7500
ORG        tp: 3 - fp: 2 - fn: 10 - tn: 3 - precision: 0.6000 - recall: 0.2308 - accuracy: 0.2000 - f1-score: 0.3334
PERSON     tp: 19 - fp: 2 - fn: 0 - tn: 19 - precision: 0.9048 - recall: 1.0000 - accuracy: 0.9048 - f1-score: 0.9500
2020-03-24 01:32:40,201 ----------------------------------------------------------------------------------------------------
Sentence: "I am from Jerusalem" - 4 Tokens
The following NER tags are found:
GPE-span [4]: "Jerusalem"


In [2]:
from presidio_evaluator.data_generator import read_synth_dataset
%reload_ext autoreload
%autoreload 2

Select data for evaluation:

In [3]:
synth_samples = read_synth_dataset("../../data/synth_dataset.json")
print(len(synth_samples))
DATASET = synth_samples

300


In [4]:
from collections import Counter
entity_counter = Counter()
for sample in DATASET:
    for tag in sample.tags:
        entity_counter[tag]+=1

In [5]:
entity_counter

Counter({'O': 4721,
         'B-PERSON': 102,
         'L-PERSON': 102,
         'U-PERSON': 72,
         'U-CREDIT_CARD': 49,
         'B-LOCATION': 47,
         'I-LOCATION': 181,
         'L-LOCATION': 47,
         'B-ORGANIZATION': 43,
         'L-ORGANIZATION': 43,
         'I-ORGANIZATION': 28,
         'U-LOCATION': 28,
         'U-EMAIL': 11,
         'U-BIRTHDAY': 4,
         'B-TITLE': 2,
         'L-TITLE': 2,
         'U-TITLE': 2,
         'B-PHONE_NUMBER': 9,
         'L-PHONE_NUMBER': 9,
         'I-PHONE_NUMBER': 18,
         'U-ORGANIZATION': 5,
         'U-IBAN': 3,
         'U-NATIONALITY': 1,
         'I-PERSON': 5})

In [6]:
DATASET[1]

Full text: Kotoya Negishi listed his top 20 songs for Entertainment Weekly and had the balls to list this song at #15. (What did he put at #1 you ask? Answer:"Tube Snake Boogie" by Frank Strauser â€“ go figure)
Spans: [Type: PERSON, value: Kotoya Negishi, start: 0, end: 14, Type: PERSON, value: Frank Strauser, start: 170, end: 184]
Tokens: [Kotoya, Negishi, listed, his, top, 20, songs, for, Entertainment, Weekly, and, had, the, balls, to, list, this, song, at, #, 15, ., (, What, did, he, put, at, #, 1, you, ask, ?, Answer:"Tube, Snake, Boogie, ", by, Frank, Strauser, â€, “, go, figure, )]
Tags: ['B-PERSON', 'L-PERSON', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PERSON', 'L-PERSON', 'O', 'O', 'O', 'O', 'O']

In [7]:
#max length sentence
max([len(sample.tokens) for sample in DATASET])

79

Select models for evaluation:

In [10]:
flair_ner = 'ner'
flair_ner_fast = 'ner-fast'
flair_ontonotes = 'ner-ontonotes-fast'
best_model = '../../models/resources/taggers/presidio-ner/best-model.pt'
final_model = '../../models/resources/taggers/presidio-ner/final-model.pt'
models = [best_model, final_model, flair_ner,flair_ner_fast,flair_ontonotes]

In [11]:
from presidio_evaluator.flair_evaluator import FlairEvaluator

for model in models:
    print("-----------------------------------")
    print("Evaluating model {}".format(model))
    flair_evaluator = FlairEvaluator(model_path=model)
    evaluation_results = flair_evaluator.evaluate_all(DATASET)
    scores = flair_evaluator.calculate_score(evaluation_results)
    
     
    print("Confusion matrix:")
    print(scores.results)

    print("Precision and recall")
    scores.print()
    errors = scores.model_errors


-----------------------------------
Evaluating model ../../models/resources/taggers/presidio-ner/best-model.pt
2020-03-24 01:35:50,525 loading file ../../models/resources/taggers/presidio-ner/best-model.pt


Evaluating <class 'presidio_evaluator.flair_evaluator.FlairEvaluator'>:   0%|          | 0/300 [00:00<?, ?it/s]

Translating entities using this dictionary: {'ORGANIZATION': 'ORG', 'COUNTRY': 'GPE', 'CITY': 'GPE', 'LOCATION': 'GPE', 'PERSON': 'PERSON', 'FIRST_NAME': 'PERSON', 'LAST_NAME': 'PERSON', 'NATION_MAN': 'GPE', 'NATION_WOMAN': 'GPE', 'NATION_PLURAL': 'GPE', 'NATIONALITY': 'GPE', 'GPE': 'GPE', 'ORG': 'ORG'}


Evaluating <class 'presidio_evaluator.flair_evaluator.FlairEvaluator'>:  95%|█████████▌| 285/300 [04:54<00:15,  1.07s/it]

2020-03-24 01:40:47,547 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?


...

Translating entities using this dictionary: {'ORGANIZATION': 'ORG', 'COUNTRY': 'GPE', 'CITY': 'GPE', 'LOCATION': 'GPE', 'PERSON': 'PERSON', 'FIRST_NAME': 'PERSON', 'LAST_NAME': 'PERSON', 'NATION_MAN': 'GPE', 'NATION_WOMAN': 'GPE', 'NATION_PLURAL': 'GPE', 'NATIONALITY': 'GPE', 'GPE': 'GPE', 'ORG': 'ORG'}


Evaluating <class 'presidio_evaluator.flair_evaluator.FlairEvaluator'>:  95%|█████████▌| 285/300 [01:27<00:05,  2.97it/s]

2020-03-24 02:09:41,814 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?


Evaluating <class 'presidio_evaluator.flair_evaluator.FlairEvaluator'>: 100%|██████████| 300/300 [01:31<00:00,  3.27it/s]

Confusion matrix:
Counter({('O', 'O'): 4538, ('PERSON', 'PERSON'): 245, ('GPE', 'O'): 166, ('O', 'WORK_OF_ART'): 90, ('ORG', 'ORG'): 73, ('O', 'DATE'): 61, ('GPE', 'GPE'): 61, ('O', 'CARDINAL'): 44, ('O', 'ORG'): 43, ('GPE', 'FAC'): 34, ('ORG', 'O'): 30, ('PERSON', 'O'): 29, ('GPE', 'CARDINAL'): 24, ('O', 'MONEY'): 14, ('O', 'NORP'): 10, ('GPE', 'LAW'): 10, ('O', 'PERSON'): 8, ('O', 'ORDINAL'): 8, ('O', 'LAW'): 8, ('ORG', 'WORK_OF_ART'): 7, ('ORG', 'PERSON'): 7, ('O', 'GPE'): 5, ('GPE', 'ORG'): 3, ('PERSON', 'ORG'): 3, ('PERSON', 'PRODUCT'): 2, ('PERSON', 'GPE'): 2, ('GPE', 'LOC'): 2, ('GPE', 'WORK_OF_ART'): 2, ('ORG', 'CARDINAL'): 1, ('GPE', 'NORP'): 1, ('O', 'LOC'): 1, ('GPE', 'PERSON'): 1, ('ORG', 'NORP'): 1})
Precision and recall
                        Entity                     Precision                        Recall
                           ORG                        61.34%                        59.84%
                           GPE                        20.07%              

Custom evaluation

#### False positives

1. Most false positive tokens:

In [12]:
errors = scores.model_errors

from presidio_evaluator import ModelEvaluator
ModelEvaluator.most_common_fp_tokens(errors)#[model_error for model_error in errors if model_error.error_type =='FP']


Most common false positive tokens:
[('the', 10), ('year', 8), ('20', 7), ('15', 7), ('1', 7), ('American', 7), ('The', 7), ('one', 7), ('Entertainment', 6), ('Weekly', 6)]
Example sentence with each FP token:
tryggvadóttir spent a year at rogers peet as the assistant to margrét tryggvadóttir, and the following year at big wheel in begonte, which later became movie gallery in 1965.
Celebrating its 10th year in Villa de Ves, Trak Auto is a 501(c)3 that invites songwriters from around the world to Texas to share the universal language of music in collaborations designed to bridge cultures, build friendships and cultivate peace.
Kotoya Negishi listed his top 20 songs for Entertainment Weekly and had the balls to list this song at #15. (What did he put at #1 you ask? Answer:"Tube Snake Boogie" by Frank Strauser â€“ go figure)
Kotoya Negishi listed his top 20 songs for Entertainment Weekly and had the balls to list this song at #15. (What did he put at #1 you ask? Answer:"Tube Snake Boogie" 

In [13]:
fps_df = ModelEvaluator.get_fps_dataframe(errors,entity=['PERSON'])
fps_df[['full_text','token','prediction']]

,full_text,token,prediction
0,Zoolander is a 2001 American action-comedy fil...,Zoolander,PERSON
1,"When they weren't singing about Hobbits, satan...",'s,PERSON
2,"When they weren't singing about Hobbits, satan...",'s,PERSON
3,This song by ex-Zombie MacMahon is a perfect e...,Zombie,PERSON
4,zoolander is a 2001 american action-comedy fil...,zoolander,PERSON
5,This song by ex-Zombie Čechová is a perfect ex...,Zombie,PERSON
6,This song by ex-Zombie Pratt is a perfect exam...,Zombie,PERSON
7,This song by ex-Zombie Mazzi is a perfect exam...,Zombie,PERSON


2. False negative examples

In [14]:
ModelEvaluator.most_common_fn_tokens(errors,n=50, entity=['PERSON'])

[('jo', 2), ('lincoln', 1), ('wormald', 1), ('Aron', 1), ('sofie', 1), ('Bojka', 1), ('Žilih', 1), ('Tiegan', 1), ('Hill', 1), ('Victoria', 1), ('Nestor', 1), ('margrét', 1), ('tryggvadóttir', 1), ('simone', 1), ('kristoffersen', 1), ('Catalina', 1), ('erick', 1), ('searlait', 1), ('Spartacus', 1), ('Innocent', 1), ('Bayer', 1), ('temirbek', 1), ('wacława', 1), ('sobczak', 1), ('Đoko', 1), ('jana', 1), ('vítková', 1), ('Guttormur', 1), ('Lóa', 1), ('Sigmundsdóttir', 1), ('van', 1), ('gameren', 1), ('Róża', 1), ('joar', 1), ('sandberg', 1)]
Token: jo, Annotation: PERSON, Full text: my name is jo van gameren but everyone calls me jo
Token: lincoln, Annotation: PERSON, Full text: from the film american graffiti (also features lincoln wormald. what's not to love?
Token: wormald, Annotation: PERSON, Full text: from the film american graffiti (also features lincoln wormald. what's not to love?
Token: Aron, Annotation: PERSON, Full text: Aron is a very sympathetic person. He's also a good lis

More FN analysis

In [15]:
fns_df = ModelEvaluator.get_fns_dataframe(errors,entity=['PERSON'])

In [16]:
fns_df[['full_text','token','annotation','prediction']]

,full_text,token,annotation,prediction
0,from the film american graffiti (also features...,lincoln,PERSON,O
1,from the film american graffiti (also features...,wormald,PERSON,O
2,Aron is a very sympathetic person. He's also a...,Aron,PERSON,O
3,sometimes people call me sofie,sofie,PERSON,O
4,"Hello, this is Eng. Bojka Žilih. Who are you?",Bojka,PERSON,O
5,"Hello, this is Eng. Bojka Žilih. Who are you?",Žilih,PERSON,O
6,The Tiegan Hill version recorded for Great Wes...,Tiegan,PERSON,PRODUCT
7,The Tiegan Hill version recorded for Great Wes...,Hill,PERSON,PRODUCT
8,I'm so jealous! said Bárður to Victoria,Victoria,PERSON,GPE
9,Nestor is a very sympathetic person. He's also...,Nestor,PERSON,O
